<a href="https://colab.research.google.com/github/heytian/d2d-oco3-tools/blob/main/nc4_plot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import rasterio
# from rasterio.transform import from_bounds
from ipywidgets import interact, IntSlider, SelectMultiple

# Base folder containing .nc4 files (replace with your own google drive shortcut to D2D shared drive)
DATA_DIR = "/content/drive/MyDrive/Shortcuts/DATA/2024-CO2-netcdfs"
OUTPUT_DIR = "/content/drive/MyDrive/Shortcuts/DATA/output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Discover years and files
files = [f for f in os.listdir(DATA_DIR) if f.endswith(".nc4")]
years = sorted(list({f.split('_')[2][:2] for f in files}))  # extract '24' from '240716'
print(f"Found {len(files)} total NC4 files.")


In [ ]:
# Display sample files per year
files_by_year = {}
for f in files:
    year = "20" + f.split('_')[2][:2]
    files_by_year.setdefault(year, []).append(f)

available_years = sorted(files_by_year.keys())
print("Available years:", available_years)

# Let user choose interactively
@interact(
    year=available_years,
    n_files=IntSlider(min=1, max=10, step=1, value=3, description='Files to process'),
)
def choose_year(year, n_files):
    chosen = files_by_year[year][:n_files]
    print(f"Will process {len(chosen)} files from {year}:")
    for f in chosen:
        print(" -", f)
    globals()['SELECTED_FILES'] = chosen


In [ ]:
# Let user preview variables in the first file
sample_file = os.path.join(DATA_DIR, files[0])
ds = xr.open_dataset(sample_file)
vars_list = list(ds.variables)
print(f"Variables found in sample file ({len(vars_list)}):")
print(vars_list[:20])  # show a subset

@interact
def choose_vars(
    csv_vars=SelectMultiple(options=vars_list, value=("xco2", "latitude", "longitude", "time"), description="CSV vars"),
    geo_var=SelectMultiple(options=vars_list, value=("xco2",), description="Geo var")
):
    globals()['CSV_VARS'] = list(csv_vars)
    globals()['GEO_VARS'] = list(geo_var)[0]
    print(f"Selected CSV vars: {CSV_VARS}")
    print(f"Selected Geo var: {GEO_VARS}")


In [ ]:
SAVE_CSV = True
SAVE_GEOTIFF = False
SAVE_PNG = False

FIXED_EXTENT = (-120, 32, -116, 36)  # Example: Los Angeles
OUT_WIDTH, OUT_HEIGHT = 360, 180

def process_nc4_file(local_path):
    filename = os.path.basename(local_path)
    csv_name = os.path.join(OUTPUT_DIR, filename.replace(".nc4", ".csv"))
    tiff_name = os.path.join(OUTPUT_DIR, filename.replace(".nc4", ".tif"))
    plot_name = os.path.join(OUTPUT_DIR, filename.replace(".nc4", ".png"))

    try:
        ds = xr.open_dataset(local_path)

        # CSV
        if SAVE_CSV:
            csv_vars_available = [v for v in CSV_VARS if v in ds.variables]
            df = ds[csv_vars_available].to_dataframe().reset_index()
            df.to_csv(csv_name, index=False)
            print(f"Saved CSV: {csv_name}")

        # GeoTIFF + PNG
        if SAVE_GEOTIFF or SAVE_PNG:
            if not all(v in ds.variables for v in ['latitude','longitude',GEO_VARS]):
                raise ValueError(f"Required variables not found in {filename}")

            lat = ds['latitude'].values
            lon = ds['longitude'].values
            data = ds[GEO_VARS].values

            lat_bins = np.linspace(FIXED_EXTENT[1], FIXED_EXTENT[3], OUT_HEIGHT)
            lon_bins = np.linspace(FIXED_EXTENT[0], FIXED_EXTENT[2], OUT_WIDTH)
            grid_data, _, _ = np.histogram2d(
                lat.flatten(), lon.flatten(),
                bins=[lat_bins, lon_bins],
                weights=data.flatten()
            )
            counts, _, _ = np.histogram2d(lat.flatten(), lon.flatten(),
                                          bins=[lat_bins, lon_bins])
            grid_data = np.divide(grid_data, counts, out=np.zeros_like(grid_data), where=counts!=0)

            if SAVE_GEOTIFF:
                transform = from_bounds(*FIXED_EXTENT, OUT_WIDTH, OUT_HEIGHT)
                with rasterio.open(
                    tiff_name, 'w', driver='GTiff',
                    height=grid_data.shape[0], width=grid_data.shape[1],
                    count=1, dtype=grid_data.dtype, crs="EPSG:4326",
                    transform=transform
                ) as dst:
                    dst.write(grid_data, 1)
                print(f"Saved GeoTIFF: {tiff_name}")

            if SAVE_PNG:
                plt.figure(figsize=(12,6))
                plt.imshow(grid_data, extent=FIXED_EXTENT, origin='lower', cmap='viridis', aspect='auto')
                plt.colorbar(label=GEO_VARS)
                plt.xlabel("Longitude")
                plt.ylabel("Latitude")
                plt.title(filename)
                plt.savefig(plot_name, bbox_inches='tight')
                plt.close()
                print(f"Saved PNG: {plot_name}")

    except Exception as e:
        print(f"Failed {local_path}: {e}")


In [ ]:
from tqdm import tqdm

for f in tqdm(SELECTED_FILES, desc=f"Processing {len(SELECTED_FILES)} files"):
    local_path = os.path.join(DATA_DIR, f)
    process_nc4_file(local_path)
